In [1]:
import numpy as np
import pandas
import os
import datetime

In [2]:
# create by price change over day

GSPC = pandas.read_csv("price_history/gspc.csv")
GSPC["Day Change"] = GSPC["Close"] - GSPC["Open"]
closing_difference = pandas.DataFrame(columns=['close_difference'])

closing_difference['close_difference'] = GSPC['Close'].diff(periods=-1)
closing_difference

closing_difference['open_deduct_previous_close'] = closing_difference['close_difference'] - GSPC["Day Change"]


GSPC["Overnight Change"] = closing_difference['open_deduct_previous_close']

# Keeping only records after 2000
relevant_record = GSPC['Date'] > "2000-01-01"
GSPC = GSPC[relevant_record]

# make csv
GSPC.to_csv("testing/GSPC.csv")

In [3]:
# industry 

with open("snp_list/snp1500_20120928.txt") as f:
    industry_dictionary = {}
    for l in f:
        industry_dictionary[l.strip().split("\t")[1]] = l.strip().split("\t")[2]

In [8]:
# create csv_from_the_txt with "Ticker", "Time", "Date",, "Time_of_day", "8K_Content"

def create_csv(ticker):
    
    # create empty dataframe with correct fields
    file_name = ticker
    ticker = ticker[:-4][6:]
    print(ticker)
    
    # Loading other relevant files
    
    price_csv_file = "price_history/" + ticker + ".csv"
    price_csv = pandas.read_csv(price_csv_file)
    GSPC = pandas.read_csv("testing/GSPC.csv")
    industry = industry_dictionary[ticker]
    print(industry)

    df = pandas.DataFrame(columns=['Ticker', 'Industry', 'Time', "Year", "Date","Time_of_day", "8K_Content"])
    
    with open(file_name) as f:
        i = 0
        lines = f.read().splitlines()
        for items in lines:
            
            if items == "<DOCUMENT>":
                text_to_parse = ""
                i += 1
            if items[:5] == "TIME:":
                time = items[5:]
                year = int(items[5:9])
#               date = items[5:9] + "-" + items[9:11] + "-" + items[11:13]
                
                t = datetime.datetime(int(items[5:9]), int(items[9:11]), int(items[11:13]), 0, 0)
                date = t.strftime('%Y-%m-%d')
                
                hour = items[13:19]
                
                if int(hour) >= 93000 and int(hour) <=160000:
                    time_of_day = "within_market"

                elif int(hour) < 93000:
                    time_of_day = "out_of_market"
                else:
                    time_of_day = "out_of_market"
                    t += datetime.timedelta(days=1)
                    date = t.strftime('%Y-%m-%d')              
                
            
            if items[:5] != "TIME:" and items[:5] != "ITEM:" and items[:5] != "EVENT" and items != "<DOCUMENT>" and items!= "</DOCUMENT>" and items[:4] !="TEXT" and items[:4] != "FILE":
                text_to_parse += items
            if items == "</DOCUMENT>":
                df.loc[i] = [ticker, industry, time, year, date, time_of_day, text_to_parse] 
                text_to_parse = ""
    
    # create by price change over day
    
    price_csv["Day Change"] = price_csv["Close"] - price_csv["Open"]
    closing_difference = pandas.DataFrame(columns=['close_difference'])

    closing_difference['close_difference'] = price_csv['Close'].diff(periods=-1)
    closing_difference

    closing_difference['open_deduct_previous_close'] = closing_difference['close_difference'] - price_csv["Day Change"]


    price_csv["Overnight Change"] = closing_difference['open_deduct_previous_close']
    
    # merging table
    Price_and_GSPC = price_csv.merge(GSPC, how='inner', on='Date', suffixes=('_stock', '_GSPC'))
    
    # join the 8k with the stock price
    everything = df.merge(Price_and_GSPC, how='inner', on='Date')
    
    #     print(everything)
    
    file_name = "testing/" + ticker + ".csv"
    
    everything = everything.drop(columns=['Unnamed: 0'])
    
    everything["Day Change_stock percentage"] = everything["Day Change_stock"]/everything["Open_stock"]
    everything["Overnight Change_stock percentage"] = everything["Overnight Change_stock"]/(everything["Open_stock"] - everything["Overnight Change_stock"])
    
    
    print(everything.columns)
    everything.to_csv(file_name)

In [9]:
# testing NVDA
# create_csv('8K-gz/NVDA.txt')

# pandas.read_csv('testing/NVDA.csv')

NVDA
Information Technology
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')


,Unnamed: 0,Ticker,Industry,Time,Year,Date,Time_of_day,8K_Content,Open_stock,High_stock,...,Open_GSPC,High_GSPC,Low_GSPC,Close_GSPC,Volume_GSPC,Adj Close_GSPC,Day Change_GSPC,Overnight Change_GSPC,Day Change_stock percentage,Overnight Change_stock percentage
0,0,NVDA,Information Technology,20030508163411,2003,2003-05-09,out_of_market,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,18.83,21.53,...,920.27,933.77,920.27,933.41,1326100000,933.41,13.14,0.00,0.134891,0.172478
1,1,NVDA,Information Technology,20030730132329,2003,2003-07-30,within_market,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,19.00,19.22,...,989.28,992.62,985.96,987.49,1391900000,987.49,-1.79,0.00,-0.007368,-0.003671
2,2,NVDA,Information Technology,20030805153324,2003,2003-08-05,within_market,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,20.99,21.47,...,982.82,982.82,964.97,965.46,1351700000,965.46,-17.36,0.00,-0.028109,0.025904
3,3,NVDA,Information Technology,20030807163643,2003,2003-08-08,out_of_market,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,16.55,16.62,...,974.12,980.57,973.83,977.59,1086600000,977.59,3.47,0.00,-0.063444,-0.142487
4,4,NVDA,Information Technology,20030822150303,2003,2003-08-22,within_market,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,19.22,19.48,...,1003.27,1011.01,992.62,993.06,1308900000,993.06,-10.21,0.00,-0.049948,0.060121
5,5,NVDA,Information Technology,20030911180413,2003,2003-09-12,out_of_market,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,18.91,19.77,...,1016.42,1019.65,1007.71,1018.63,1236700000,1018.63,2.21,0.00,0.023268,-0.008390
6,6,NVDA,Information Technology,20030917150125,2003,2003-09-17,within_market,UNITED STATES SECURITIES AND EXCHANGE COMMISSI...,19.74,20.23,...,1029.32,1031.34,1024.53,1025.97,1338210000,1025.97,-3.35,0.00,0.005572,0.006116
7,7,NVDA,Information Technology,20031106162141,2003,2003-11-07,out_of_market,Item 12. Results of Operations and Financial...,21.66,22.06,...,1058.05,1062.39,1052.17,1053.21,1440500000,1053.21,-4.84,0.00,0.004155,0.192075
8,8,NVDA,Information Technology,20040212163736,2004,2004-02-13,out_of_market,Item 12. Results of Operations and Financia...,22.90,23.89,...,1152.11,1156.88,1143.24,1145.81,1329200000,1145.81,-6.30,0.00,0.017467,-0.026361
9,9,NVDA,Information Technology,20040426154610,2004,2004-04-26,within_market,Item 4. Change in Registrant's Certifying Ac...,23.29,23.39,...,1140.60,1145.08,1132.91,1135.53,1290600000,1135.53,-5.07,0.00,-0.041219,0.002151


In [10]:
list_of_txt = os.listdir("8K-gz") 

for i in list_of_txt:
    file_directory = "8K-gz/" + i
    try:
        create_csv(file_directory)
    except:
        pass

EPIQ
Information Technology
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
KMB
Consumer Staples
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype

GS
Financials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
CYN
Financials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
EL
Consum

PM
Consumer Staples
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
ATK
Industrials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
CN

Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
HSP
Health Care
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
DELL
Information Techn

Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
GE
Industrials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
SCSS
Consumer Discretio

...

Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
THOR
Health Care
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
IT
Information Techno

Financials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
FSS
Industrials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Time_of_day', '8K_Content',
       'Date', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
MRCY
Inform

MINI
Industrials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
VMI
Industrials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
HF
Fi

COLB
Financials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
BRS
Energy
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')
NDAQ
Financ

KOP
Materials
Index(['Ticker', 'Industry', 'Time', 'Year', 'Date', 'Time_of_day',
       '8K_Content', 'Open_stock', 'High_stock', 'Low_stock', 'Close_stock',
       'Volume_stock', 'Adj Close_stock', 'Day Change_stock',
       'Overnight Change_stock', 'Open_GSPC', 'High_GSPC', 'Low_GSPC',
       'Close_GSPC', 'Volume_GSPC', 'Adj Close_GSPC', 'Day Change_GSPC',
       'Overnight Change_GSPC', 'Day Change_stock percentage',
       'Overnight Change_stock percentage'],
      dtype='object')


In [11]:
csv_list = []

list_of_csv = os.listdir("testing/") 
for i in list_of_csv:
    i = 'testing/' + i
    try:
        csv_list.append(pandas.read_csv(i))
    except:
        pass

merged_csv = pandas.concat(csv_list, ignore_index=True)

# drop unecessary columns
merged_csv = merged_csv.drop(columns=['Close', 'Open', 'Day Change', 'Low', 'High', 'Overnight Change', 'Volume','Adj Close', 'Unnamed: 0'])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [12]:
# # adding back the year.....
# merged_csv['Year'] = merged_csv['Date']
merged_csv.head()

,8K_Content,Adj Close_GSPC,Adj Close_stock,Close_GSPC,Close_stock,Date,Day Change_GSPC,Day Change_stock,Day Change_stock percentage,High_GSPC,...,Open_stock,Overnight Change_GSPC,Overnight Change_stock,Overnight Change_stock percentage,Ticker,Time,Time_of_day,Volume_GSPC,Volume_stock,Year
0,==============================================...,953.99,12.56,953.99,13.04,2002-07-03,5.90,0.71,0.057583,954.30,...,12.33,0.0,-0.23,-0.018312,CSCO,2.002070e+13,out_of_market,1.527800e+09,77171300.0,2002.0
1,==============================================...,834.60,10.94,834.60,11.36,2002-08-05,-29.64,-0.22,-0.018998,864.24,...,11.58,0.0,-0.31,-0.026072,CSCO,2.002081e+13,within_market,1.425500e+09,83088100.0,2002.0
2,Table of ContentsDate of report (Date of earli...,937.43,14.18,937.43,14.73,2002-08-20,-13.27,0.06,0.004090,950.70,...,14.67,0.0,-0.05,-0.003397,CSCO,2.002082e+13,out_of_market,1.308500e+09,60171900.0,2002.0
3,Table of Contents Table of ContentsItems 1 th...,843.32,11.56,843.32,12.00,2002-09-19,-26.14,0.04,0.003344,869.46,...,11.96,0.0,-0.54,-0.043200,CSCO,2.002092e+13,out_of_market,1.524000e+09,80020200.0,2002.0
4,Table of ContentsDate of report (Date of earli...,882.50,11.31,882.50,11.75,2002-10-24,-13.64,0.27,0.023519,902.94,...,11.48,0.0,0.22,0.019538,CSCO,2.002102e+13,out_of_market,1.700570e+09,122749300.0,2002.0


In [62]:
# # EDA
# # merged_csv[merged_csv[Time_of_day] == "out_of_market"]

# # merged_csv.columns
# merged_csv.head()

# # merged_csv[merged_csv.Time_of_day == "out_of_market"]["Time_of_day"].value_count()

,8K_Content,Adj Close_GSPC,Adj Close_stock,Close_GSPC,Close_stock,Date,Day Change_GSPC,Day Change_stock,Day Change_stock percentage,High_GSPC,...,Open_stock,Overnight Change_GSPC,Overnight Change_stock,Overnight Change_stock percentage,Ticker,Time,Time_of_day,Volume_GSPC,Volume_stock,Year
0,==============================================...,953.99,12.56,953.99,13.04,2002-07-03,5.90,0.71,0.057583,954.30,...,12.33,0.0,-0.23,-0.018312,CSCO,2.002070e+13,out_of_market,1.527800e+09,77171300.0,2002-07-03
1,==============================================...,834.60,10.94,834.60,11.36,2002-08-05,-29.64,-0.22,-0.018998,864.24,...,11.58,0.0,-0.31,-0.026072,CSCO,2.002081e+13,within_market,1.425500e+09,83088100.0,2002-08-05
2,Table of ContentsDate of report (Date of earli...,937.43,14.18,937.43,14.73,2002-08-20,-13.27,0.06,0.004090,950.70,...,14.67,0.0,-0.05,-0.003397,CSCO,2.002082e+13,out_of_market,1.308500e+09,60171900.0,2002-08-20
3,Table of Contents Table of ContentsItems 1 th...,843.32,11.56,843.32,12.00,2002-09-19,-26.14,0.04,0.003344,869.46,...,11.96,0.0,-0.54,-0.043200,CSCO,2.002092e+13,out_of_market,1.524000e+09,80020200.0,2002-09-19
4,Table of ContentsDate of report (Date of earli...,882.50,11.31,882.50,11.75,2002-10-24,-13.64,0.27,0.023519,902.94,...,11.48,0.0,0.22,0.019538,CSCO,2.002102e+13,out_of_market,1.700570e+09,122749300.0,2002-10-24


In [64]:
# create the final 
merged_csv.to_csv('final.csv')
merged_csv_only_overnight = merged_csv[final["Time_of_day"] == "out_of_market"]
# final = pandas.read_csv('final.csv')

In [78]:
print(merged_csv_only_overnight["Overnight Change_stock percentage"].mean())
print(merged_csv_only_overnight["Overnight Change_stock percentage"].median())
print(merged_csv_only_overnight["Overnight Change_stock percentage"].std())
print(merged_csv_only_overnight["Overnight Change_stock percentage"].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]))

0.012371812184204477
0.0
3.877378559741765
0.1   -0.025516
0.2   -0.009880
0.3   -0.004530
0.4   -0.001493
0.5    0.000000
0.6    0.002494
0.7    0.005608
0.8    0.011364
0.9    0.027523
Name: Overnight Change_stock percentage, dtype: float64
